# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4835, uptime 0:00:34
mini_beamline                    RUNNING   pid 4836, uptime 0:00:34
random_walk                      RUNNING   pid 4837, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4838, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4839, uptime 0:00:34
simple                           RUNNING   pid 4840, uptime 0:00:34
thermo_sim                       RUNNING   pid 4841, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4842, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 20:11:42
Persistent Unique Scan ID: 'a0f0fbcf-d6ac-420f-a7f5-32ef36fbc187'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:11:42.3 |     -1.000 |      98394 |


|         2 | 20:11:42.3 |     -0.500 |      97120 |
|         3 | 20:11:42.3 |      0.000 |      98486 |
|         4 | 20:11:42.3 |      0.500 |      97743 |
|         5 | 20:11:42.3 |      1.000 |      96815 |


+-----------+------------+------------+------------+
generator scan ['a0f0fbcf'] (scan num: 3)





('a0f0fbcf-d6ac-420f-a7f5-32ef36fbc187',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/a0f0fbcf-d6ac-420f-a7f5-32ef36fbc187-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-18 20:11:42
Persistent Unique Scan ID: '29e1bd58-c296-4028-9e67-e54175bb0398'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:11:42.8 |          -1 | 1063350466 |


|         2 | 20:11:43.0 |          -0 | 1056086744 |


|         3 | 20:11:43.1 |           0 | 1053929803 |


|         4 | 20:11:43.3 |           0 | 1053284249 |


|         5 | 20:11:43.4 |           1 | 1054899990 |


+-----------+------------+-------------+------------+
generator scan ['29e1bd58'] (scan num: 4)





('29e1bd58-c296-4028-9e67-e54175bb0398',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-18 20:11:43
Persistent Unique Scan ID: 'c5e579aa-15b4-4f06-9f0f-82a884ff8308'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:11:43.9 | 1074071561 |


|         2 | 20:11:44.8 | 1127317029 |


|         3 | 20:11:45.8 | 1110379667 |


...

|        28 | 20:12:10.9 | 1056454549 |


|        29 | 20:12:11.9 | 1077405920 |


|        30 | 20:12:12.9 | 1126116154 |


+-----------+------------+------------+
generator count ['c5e579aa'] (scan num: 5)





('c5e579aa-15b4-4f06-9f0f-82a884ff8308',)